<a href="https://colab.research.google.com/github/umangja/Music-Playlist-Generation-And-Shuffling/blob/master/MusicReccommendationSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Important Note**

1. First of all mount your G drive to colab.

2. Then upload data file on your gdrive.

3. Define path file address accordingly.



#**Mounting Gdrive**

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# **Installing Required Libs**

In [0]:
!pip3 install fuzzywuzzy

# **Import Libs**

In [0]:
import pandas as pd
from scipy.spatial.distance import cosine
import numpy as np
from scipy.sparse import csr_matrix
from  sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
import  string
from fuzzywuzzy import fuzz


/usr/local/lib/python3.6/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# **Setting Default Value**

In [0]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# **Defining Path Variable**

In [0]:
usersha1_artmbid_artname_plays_path = "/content/drive/My Drive/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv"
usersha1_profile_path               = "/content/drive/My Drive/lastfm-dataset-360K/usersha1-profile.tsv"
CSR_save_file_name                  = "first.npz"

# **Importing last.fm dataset**

In [0]:
user_data = pd.read_table(usersha1_artmbid_artname_plays_path,
                          header = None, nrows = 2e7,
                          names = ['users', 'musicbrainz-artist-id', 'artist-name', 'plays'],
                          usecols = ['users', 'artist-name', 'plays'])

user_profiles = pd.read_table(usersha1_profile_path  ,
                          header = None,
                          names = ['users', 'gender', 'age', 'country', 'signup'],
                          usecols = ['users', 'country'])


# **Printing dataset**

In [0]:
print(user_data)
print("\n\n")
print(user_profiles)

                                             users           artist-name  plays
0         00000c289a1829a808ac09c00daf10bc3c4e223b       betty blowtorch   2137
1         00000c289a1829a808ac09c00daf10bc3c4e223b             die Ärzte   1099
2         00000c289a1829a808ac09c00daf10bc3c4e223b     melissa etheridge    897
3         00000c289a1829a808ac09c00daf10bc3c4e223b             elvenking    717
4         00000c289a1829a808ac09c00daf10bc3c4e223b  juliette & the licks    706
...                                            ...                   ...    ...
17535650                              sep 20, 2008            turbostaat     12
17535651                              sep 20, 2008         cuba missouri     11
17535652                              sep 20, 2008       little man tate     11
17535653                              sep 20, 2008             sigur rós     10
17535654                              sep 20, 2008            the smiths     10

[17535655 rows x 3 columns]



        

# **Removing very unpopular artists**
removing all those rows from which contain null value at artist-name column


In [0]:
if user_data['artist-name'].isnull().sum() > 0: user_data = user_data.dropna(axis = 0, subset = ['artist-name'])

#**Some more modification**

In [0]:
artist_plays = (user_data.
     groupby(by = ['artist-name'])['plays'].
     sum().
     reset_index().
     rename(columns = {'plays': 'total_artist_plays'})
     [['artist-name', 'total_artist_plays']]
    )

# print(artist_plays)


**Merging user data with artist play**

In [0]:
user_data_with_artist_plays = user_data.merge(artist_plays, left_on = 'artist-name', right_on = 'artist-name', how = 'left')

# print(user_data_with_artist_plays.head())

**user data for popular artist**

In [0]:
#artist popularity threshold (change as per pref)
popularity_threshold = 100000
user_data_popular_artists = user_data_with_artist_plays.query('total_artist_plays >= @popularity_threshold')
user_data_popular_artists.head()

,users,artist-name,plays,total_artist_plays
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099,3704875
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897,180391
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717,410725
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691,13547741
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507,1004743


**US users only for less data **

In [0]:
combined = user_data_popular_artists.merge(user_profiles, left_on = 'users', right_on = 'users', how = 'left')
usa_data = combined.query('country == \'United States\'')

if not usa_data[usa_data.duplicated(['users', 'artist-name'])].empty:
    initial_rows = usa_data.shape[0]
    usa_data = usa_data.drop_duplicates(['users', 'artist-name'])
    current_rows = usa_data.shape[0]

usa_data.head()

,users,artist-name,plays,total_artist_plays,country
134,00007a47085b9aab8af55f52ec8846ac479ac4fe,devendra banhart,456,2366807,United States
135,00007a47085b9aab8af55f52ec8846ac479ac4fe,boards of canada,407,6115545,United States
136,00007a47085b9aab8af55f52ec8846ac479ac4fe,cocorosie,386,2194862,United States
137,00007a47085b9aab8af55f52ec8846ac479ac4fe,aphex twin,213,4248296,United States
138,00007a47085b9aab8af55f52ec8846ac479ac4fe,animal collective,203,3495537,United States


In [0]:
wide_artist_data = usa_data.pivot(index = 'artist-name', columns = 'users', values = 'plays').fillna(0)
# print(wide_artist_data)
wide_artist_data_sparse = csr_matrix(wide_artist_data.values)
# print(wide_artist_data_sparse)


# **Some Usefull Methods**

In [0]:
#save model
def save_sparse_csr(filename,array):
    np.savez(filename,data = array.data ,indices=array.indices,
             indptr =array.indptr, shape=array.shape )

def load_sparse_csr(filename):
    loader = np.load(filename)
    return csr_matrix((loader['data'], loader['indices'], loader['indptr']),
                         shape = loader['shape'])
    
def  print_artist_recommendations (query_artist, artist_plays_matrix, knn_model, k):

  #  query_artist: query artist name
    
    query_index = None
    ratio_tuples = []
    recommended_artists = []
    
    for i in artist_plays_matrix.index:
        ratio = fuzz.ratio(i.lower(), query_artist.lower())
        if ratio >= 75:
            current_query_index = artist_plays_matrix.index.tolist().index(i)
            ratio_tuples.append((i, ratio, current_query_index))
    
    print ('Possible matches: {0}\n'.format([(x[0], x[1]) for x in ratio_tuples]))
    
    try:
        query_index = max(ratio_tuples, key = lambda x: x[1])[2] # get the index of the best artist match in the data
    except:
        print ('Your artist didn\'t match any artists in the data. Try again')
        return None
    
    distances, indices = knn_model.kneighbors(artist_plays_matrix.iloc[query_index, :].values.reshape(1, -1), n_neighbors = k + 1)

    for i in range(0, len(distances.flatten())):
        if i == 0:
            print ('Recommendations for {0}:\n'.format(artist_plays_matrix.index[query_index]))
        else:
            recommended_artists.append(artist_plays_matrix.index[indices.flatten()[i]])
            print ('{0}: {1}, with distance of {2}:'.format(i, artist_plays_matrix.index[indices.flatten()[i]], distances.flatten()[i]))

    return recommended_artists

# **Saving sparse CSR**

In [0]:
save_sparse_csr(CSR_save_file_name, wide_artist_data_sparse)

# **Training**

In [0]:
#fit model
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(wide_artist_data_sparse)


NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

# **Print Top 10 recommendations**

In [0]:
recommended_artists = print_artist_recommendations('bob dylan', wide_artist_data, model_knn, k = 10)

print(recommended_artists)

Possible matches: [('bob dylan', 100)]

Recommendations for bob dylan:

1: the band, with distance of 0.6894347031604409:
2: grateful dead, with distance of 0.732429870951373:
3: neil young, with distance of 0.7349833775494925:
4: the rolling stones, with distance of 0.7363390873659932:
5: the beatles, with distance of 0.7475805124322863:
6: the velvet underground, with distance of 0.8000663230166598:
7: led zeppelin, with distance of 0.8093019262072221:
8: jimi hendrix, with distance of 0.8136566190704223:
9: the kinks, with distance of 0.8179323589409717:
10: the allman brothers band, with distance of 0.8183530325911672:
['the band', 'grateful dead', 'neil young', 'the rolling stones', 'the beatles', 'the velvet underground', 'led zeppelin', 'jimi hendrix', 'the kinks', 'the allman brothers band']


# **Playlist shuffling**

## **Fisher Yates**

In [0]:
def Fisher_yates(recommended_artists):
    import random 
    for i in range(len(recommended_artists)-1, 0, -1): 
        j = random.randint(0, i + 1)  
        if(j>=len(recommended_artists) or i>=len(recommended_artists)):
             print("NOT GOOD")
        recommended_artists[i], recommended_artists[j] = recommended_artists[j], recommended_artists[i]  
    return recommended_artists

In [0]:
recommended_artists_acc_to_fisher_yates = Fisher_yates(recommended_artists)
for idx,row in enumerate(recommended_artists_acc_to_fisher_yates):
    print(str(idx)+" : "+row+"\n")
# print(recommended_artists_acc_to_fisher_yates)

0 : the rolling stones

1 : the velvet underground

2 : the allman brothers band

3 : the beatles

4 : neil young

5 : the band

6 : jimi hendrix

7 : the kinks

8 : led zeppelin

9 : grateful dead



## **Etilist Shuffling**

In [0]:
def Etilist_Shuffle(items,inequality):
    weights = np.power(np.linspace(1, 0, num=len(items), endpoint=False),inequality)
    weights = weights / np.linalg.norm(weights, ord=1)
    return np.random.choice(items, size=len(items), replace=False, p=weights)

In [0]:
inequality=5
Rec_ArtistEtilist = Etilist_Shuffle(recommended_artists,inequality)
for idx,row in enumerate(Rec_ArtistEtilist):
    print(str(idx)+" : "+row+"\n")

0 : the beatles

1 : the rolling stones

2 : the velvet underground

3 : the allman brothers band

4 : neil young

5 : the kinks

6 : the band

7 : jimi hendrix

8 : led zeppelin

9 : grateful dead



# **Stay Safe**